In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
import librosa as lb
from sklearn.metrics import confusion_matrix

In [24]:
label_emotion = []

for file in os.listdir('./AudioWAV/') :
    part=file.split('_')
    if part[2] == 'SAD':
        label_emotion.append(0)
    elif part[2] == 'ANG':
        label_emotion.append(1)
    elif part[2] == 'DIS':
        label_emotion.append(2)
    elif part[2] == 'FEA':
        label_emotion.append(3)
    elif part[2] == 'HAP':
        label_emotion.append(4)
    elif part[2] == 'NEU':
        label_emotion.append(5)
    else:
        label_emotion.append(6)

In [26]:
def extract_feature(X, sample_rate):

  result = np.array([])
  #MFCC
  mfcc = np.mean(lb.feature.mfcc(y=X, sr=sample_rate).T,axis = 0)
  result = np.hstack((result,mfcc))

  #Chroma
  stft = np.abs(lb.stft(X))
  chroma = np.mean(lb.feature.chroma_stft(S=stft, sr = sample_rate).T,axis=0)
  result = np.hstack((result,chroma))
  #Mel
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
  result = np.hstack((result,mel))

  return result

In [31]:
l,r = lb.load('./AudioWAV/1014_TSI_DIS_XX.wav')

In [32]:
speech_list = []
for file in os.listdir('./AudioWAV/') :
    audio, sample_rate = lb.load('./AudioWAV/'+file)
    speech_list.append(extract_feature(audio,sample_rate))

1014_TSI_DIS_XX.wav
1002_IEO_DIS_MD.wav
1041_TSI_HAP_XX.wav
1069_DFA_ANG_XX.wav


/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[-4.4501163e-03 -5.5457731e-03 -5.8122263e-03 ... -4.6996935e-05
 -5.9750044e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[-3.2579229e-04 -5.9018063e-04 -3.1577528e-04 ... -9.9059164e-08
  8.5180396e-09  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[7.8329258e-03 9.8130871e-03 9.0365307e-03 ... 3.5775556e-05 2.9189214e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3

1054_TSI_HAP_XX.wav
1041_WSI_NEU_XX.wav
1053_IOM_SAD_XX.wav
1023_DFA_FEA_XX.wav


/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[1.4567267e-02 1.7526697e-02 1.5296278e-02 ... 3.3806682e-05 2.8989387e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[ 6.3436764e-06 -2.3850719e-04  1.6984079e-04 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[-1.2256064e-03 -1.6510818e-03 -7.8655104e-04 ... -6.4436201e-05
 -2.0724690e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3

1062_IWL_FEA_XX.wav
1066_WSI_SAD_XX.wav
1085_IWL_HAP_XX.wav
1036_ITH_SAD_XX.wav


...

/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[-1.2390454e-03 -1.3632650e-03 -1.3363947e-03 ...  2.2380455e-05
 -5.3302424e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[0.00075348 0.00063436 0.00039126 ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[-3.6120934e-03 -4.6425136e-03 -3.9801998e-03 ... -8.3732841e-05
 -4.5019740e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: Fu

1012_DFA_FEA_XX.wav
1061_IEO_SAD_LO.wav
1071_IEO_DIS_HI.wav
1037_IWW_ANG_XX.wav


/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[-2.1840485e-04  3.4668253e-06  7.6472270e-04 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[ 1.4542189e-03  2.1915780e-03  2.3532682e-03 ... -2.0796090e-06
 -1.4167557e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[0.00240344 0.00264117 0.0022412  ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)


1074_MTI_HAP_XX.wav
1004_IWL_SAD_XX.wav
1003_IEO_HAP_LO.wav
1067_IEO_NEU_XX.wav
1025_ITH_ANG_XX.wav
1066_TAI_NEU_XX.wav


/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[-0.00460963 -0.00575664 -0.00508036 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[3.2045846e-03 3.4764598e-03 2.7891328e-03 ... 1.5534417e-04 3.7929083e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  5.2174883e-06
 -3.1339514e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(lb.feature.melspectrogram(X, sr = sample_rate).T,axis=0)
/tmp/ipykernel_19560/3907510176.py:13: FutureWar

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(speech_list,label_emotion, train_size = 0.8)

In [54]:
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [60]:
model = Sequential()

model.add(layers.Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1],1)))
model.add(layers.MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(layers.Flatten())
model.add(layers.Dense(6))
model.add(layers.Activation('softmax'))

In [66]:
model=Sequential()
model.add(layers.Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(layers.MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(layers.Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(layers.Flatten())
model.add(layers.Dense(units=32, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(units=6, activation='softmax'))
model.compile(optimizer = 'rmsprop' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 160, 256)          1536      
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 80, 256)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 80, 256)           327936    
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 40, 256)          0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 40, 128)           163968    
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 20, 128)         

In [61]:
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

In [62]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 160, 128)          768       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 80, 128)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 80, 128)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 80, 64)            41024     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 40, 64)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 2560)             

In [67]:
rlrp = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
history=model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[rlrp])

Epoch 1/50
187/187 [==============================] - 8s 39ms/step - loss: 1.8858 - accuracy: 0.1740 - val_loss: 1.7913 - val_accuracy: 0.1632 - lr: 0.0010
Epoch 2/50
187/187 [==============================] - 8s 44ms/step - loss: 1.7908 - accuracy: 0.1670 - val_loss: 1.7909 - val_accuracy: 0.1632 - lr: 0.0010
Epoch 3/50
187/187 [==============================] - 9s 48ms/step - loss: 1.7905 - accuracy: 0.1609 - val_loss: 1.7908 - val_accuracy: 0.1632 - lr: 0.0010
Epoch 4/50
187/187 [==============================] - 9s 48ms/step - loss: 1.7904 - accuracy: 0.1727 - val_loss: 1.7908 - val_accuracy: 0.1632 - lr: 0.0010
Epoch 5/50
 97/187 [==============>...............] - ETA: 4s - loss: 1.7933 - accuracy: 0.1685

KeyboardInterrupt: 